### DNN 기반 회귀 모델 구현 + 학습진행 모니터링 <hr>
- 데이터셋 : iris.csv
- 피쳐/속성 : 3개 Sepal_Length, Sepan_Width, Petal_ Length
- 타겟/라벨 : 1개 Peatal_Width
- 학습방벙 : 지도학습 > 회귀
- 알고리즘 : 인공신경망 (ANN) -> MLP(Multi Layer Percetron),  심층신경망(DNN)(은닉층 多)
- 프레임워크 : Pytorch
---
- 모니터링
    - 기준 : 검증테이터셋의 loss 와 score
    - 평가 : 학습데이텅셋의 loss와 score와 비교해서 학습 중단 여부를 결정
    - 선택 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장여부 또는 모델 전체 저장
---

[1] 모듈 로딩 및 데이터 준비  <hr>

In [208]:
# 모듈로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# - Data관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [209]:
# 활용 패키지 버전 체크 - 사용자 정의 함수로 구현할것!
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytorch v.2.4.1
Pandas v.2.0.3


In [210]:
## 데이터로딩
DATA_FILE='../../MachineLearning/data/iris.csv'

## CSV >> DataFrame
irisDF=pd.read_csv(DATA_FILE)

## 확인
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스목적 : iris 데이터를 학습 및 추론 목적
- 클래스이름 : IrisRegModel
- 부모클래스 : nn.Module
- 매개변수 : 층별 입출력 갯수 고정하므로 필요 없음
- 속성/필드 : featruesDF, targetDF, n_rows, n_features
- 기능/역할 <필수 >
    -  __ init __() : 모델 구조 설정 
    - forward() : 순방향 학습 <- 오버라이딩(overriding)
- 클래스구조
    - 입력층 : 입력 3개 / 출력 10개 (퍼셉트론/뉴런 10개 존재)
    - 은닉층 : 입력 10개 / 출력 30개 (퍼셉트론/뉴런 30개 존재) 
    - 출력층 : 입력 30개 / 출력 1개 (너비값)
- 손실함수/ 활성화함수 
    - 클래스형태 => nn. 아래  
        - ex) nn.ReLu << __init__() 에 사용
    - 함수 형태 => torch.nn.functional 아래
        - ex) F.relu << def forward() 에 사용


In [211]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(3,10)
        self.hd_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,1)

    # 순방향 학습 진행 메서드
    def forward(self,x):
        # - 입력층
        y=self.in_layer(x)        # f1_1w1_1+f1_2w1_2+f1_3w1_3+b << 이런식 10개
        y=F.relu(y)

        # - 은닉층 : 10개의 숫자값 (>=0)
        y=self.hd_layer(y)        # f2_1w2_1+f2_2w2_2+....f2_10w2_10 +b << 이런식 30개
        y=F.relu(y)

        # - 출력층 : 30개의 숫자값 (>=0) - 회귀이므로 바로 반환
        return self.out_layer(y)         # f3_1w3_1+.....f3_30w3_30+b << 1 개

    

In [212]:
### [테스트] 모델 인스턴스 생성
model=IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [213]:
### [테스트] 모델 확인
summary(model,input_size=(17,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [17, 1]                   --
├─Linear: 1-1                            [17, 10]                  40
├─Linear: 1-2                            [17, 30]                  330
├─Linear: 1-3                            [17, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 3개
- 타겟개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드 
    - __ init __(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    - __ len __(self) : 데이터의 개수 반환
    - __ getitem __(self,index) : 특정 인덱스의 피쳐와 타겟 반환

In [214]:
class IrisDataset(Dataset):

    def __init__(self,featureDF,targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]      # 필수아님
        self.n_features=featureDF.shape[1]  # 필수아님


    def __len__(self):
        return self.n_rows
        
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS
    

In [215]:
## 데이터셋 인스턴스 생성

featureDF=irisDF[irisDF.columns[:3]]        # 2D (150,3)
targetDF=irisDF[irisDF.columns[[3]]]        # 2D (150,1)

# - 커스템데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF,targetDF)

[4] 학습 준비
- 학습횟수 : EPOCH <- 처음부터 끝까지 공부하는 횟수
- 배치크기 : BATCH_SIZE <- 한번에 학습할 데이터셋 양
- 위치지정 : DEVICE <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학습률(LR, learning Rate) : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1

In [216]:
### 학습 진행 관련 설정 값 
EPOCH = 100
BATCH_SIZE= 10
# BATCH_CNT=irisDF.shape[0]/BATCH_SIZE      = len(irisDL)
DEVICE ='cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화, 손실함수 (, 성능지표)

In [217]:
# 모델 인스턴스
model= IrisRegModel()

# 데이터셋 인스턴스
# # 학습,검증,테스트 데이터 분리
X_train, X_test, y_train, y_test=train_test_split(featureDF,targetDF,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=1)
print(f' X_train : {X_train.shape}, X_val : {X_val.shape} , X_test : {X_test.shape}')
print(f' y_train : {y_train.shape}, y_val : {y_val.shape} , y_test : {y_test.shape}')



trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)

# 데이터로더 인스턴스
trainDL=DataLoader(trainDS,batch_size=BATCH_SIZE)

 X_train : (84, 3), X_val : (28, 3) , X_test : (38, 3)
 y_train : (84, 1), y_val : (28, 1) , y_test : (38, 1)


In [218]:
## [테스트] 데이터 로더
for feature,target in trainDL:
    print(feature.shape,target.shape)
    

torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([4, 3]) torch.Size([4, 1])


In [219]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(),lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE, ...
regLoss=MeanSquaredError()

[5] 학습진행

In [220]:
## 학습의 효과 확인 손실값과 성능 평가값 저장 필요
LOSS_HISTORY,SCORE_HISTORY=[[],[]],[[],[]]

# 학습 모니터링/스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY활용
# 임계기준 : 10번 (내가설정)
BREAK_CNT= 0


# 학습 모드로 모델 설정
 
for epoch in range(EPOCH):
    model.train()  ## 학습모드 켜기! 에포크 단위로 학습과 검증 진행시 for문안에서 껐다 켯다 해야함! 

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total=0
    score_total=0
    for featureTS,targetTS in trainDL:

        # 학습 진행
        pre_y=model(featureTS)

        # 손실 계산
        loss=regLoss(pre_y,targetTS)
        loss_total+=loss.item()

        # 성능평가 계산
        score=R2Score()(pre_y,targetTS)
        score_total+=score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval() # 검증모드 켜기!
    with torch.no_grad():
        # 검증데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        # 평가
        pre_val = model(val_featureTS)
        #손실
        val_loss=regLoss(pre_val,val_targetTS)
        # 성능평가
        val_score=R2Score()(pre_val,val_targetTS)
    
    # 손실값과 성능 평가값 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))

    LOSS_HISTORY[1].append(val_loss)
    SCORE_HISTORY[1].append(val_score)
    print(f'[{epoch+1}/{EPOCH}]')


    # 학습 진행 모니터링/스케쥴링- 검증기준 
    # Loss 기준
    if len(LOSS_HISTORY[0])>=2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]:
            BREAK_CNT+=1

    # Score 기준
    # if len(SCORE_HISTORY[0])>=2:
    #     if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]:
    #         BREAK_CNT+=1
    # loss나 score 둘중 하나만 사용

    ## 학습 중단 여부 설정
    if BREAK_CNT>10:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break

[1/100]
[2/100]
[3/100]
[4/100]
[5/100]
[6/100]
[7/100]
[8/100]
[9/100]
[10/100]
[11/100]
[12/100]
[13/100]
[14/100]
[15/100]
[16/100]
[17/100]
[18/100]
[19/100]
[20/100]
[21/100]
[22/100]
[23/100]
[24/100]
[25/100]
[26/100]
[27/100]
[28/100]
[29/100]
[30/100]
[31/100]
[32/100]
[33/100]
[34/100]
[35/100]
[36/100]
[37/100]
[38/100]
[39/100]
[40/100]
[41/100]
[42/100]
[43/100]
[44/100]
[45/100]
[46/100]
[47/100]
[48/100]
[49/100]
[50/100]
[51/100]
[52/100]
[53/100]
[54/100]
[55/100]
[56/100]
[57/100]
성능 및 손실 개선이 없어서 학습 중단


In [221]:
LOSS_HISTORY

[[1.1511929896142747,
  0.6296300623151991,
  0.3788959350850847,
  0.30510979725254905,
  0.28684213426378036,
  0.2624758283297221,
  0.24092408021291098,
  0.22452214856942496,
  0.2071435269382265,
  0.1864429364601771,
  0.16214663783709207,
  0.13445253918568292,
  0.10818733771642049,
  0.08698991189400355,
  0.0685868544711007,
  0.05458753224876192,
  0.045295077893469066,
  0.03948400624924236,
  0.03613893687725067,
  0.034421656280756,
  0.033549586310982704,
  0.03303507715463638,
  0.03268662571079201,
  0.032409210999806724,
  0.03214946803119448,
  0.03190135728153917,
  0.03161004475421376,
  0.03144858218729496,
  0.03117347363796499,
  0.030957663017842505,
  0.030798699085911114,
  0.030521793704893854,
  0.030388870379991002,
  0.030146748655372195,
  0.02998190393878354,
  0.029845143978794415,
  0.029614841979410913,
  0.029530631171332464,
  0.029329484328627586,
  0.029208921103013888,
  0.029056585497326322,
  0.028963238207830325,
  0.028791043079561658,
  0.

- 모델 저장 방법 </hr>
- 방법1 : 모델 파라미터만 저장
- 방법2 : 모델 설계 구조 및 파라미터 까지 모두 저장

In [222]:
## 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[-0.1365, -0.5364, -0.3836],
                      [ 0.5722, -0.5604, -0.2501],
                      [ 0.4147,  0.4546, -0.0468],
                      [-0.0063, -0.1399,  0.4483],
                      [ 0.2955,  0.5401, -0.3780],
                      [ 0.5136, -0.3631, -0.3699],
                      [-0.2814,  0.0172,  0.3477],
                      [ 0.4181, -0.1767,  0.4972],
                      [-0.1799,  0.0568, -0.3362],
                      [ 0.2840,  0.5489,  0.6405]])),
             ('in_layer.bias',
              tensor([-0.4536, -0.0729,  0.3815, -0.3912, -0.1116,  0.1785, -0.4195, -0.4432,
                       0.5161, -0.3506])),
             ('hd_layer.weight',
              tensor([[ 4.8242e-02,  1.7043e-01,  2.8597e-01,  5.2063e-02, -7.9224e-02,
                       -1.3583e-01,  3.8102e-03,  2.3857e-01, -1.4904e-01, -1.7222e-01],
                      [-2.7001e-01, -2.8398e-01, -2.0918e-01, -2.4336e-01, -

- [방법1] 모델파라미터 즉, 층별 가중치와 절편들

In [223]:
### models 폴더 아래 프로젝트 폴더 아래 모델파일 저장
import os

# 저장경로
SAVE_PATH='../models/iris/'
# 저장 파일명
SAVE_FILE='model_train_wbs.pth'


In [224]:
# 경로상 폴더 존재 여부 체크

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)  # 폴더/폴더/... 하위폴더까지 생성

In [225]:
# 모델 저장
torch.save(model.state_dict(),SAVE_PATH+SAVE_FILE)

#### 모델 즉, 가중치와 절편 로딩
- [1] 가중치와 절편 객체로 로딩
- [2] 모델의 state_dict 속성에 저장

In [228]:
# 읽기
wbTS=torch.load(SAVE_PATH+SAVE_FILE,weights_only=False)
print(wbTS)

OrderedDict([('in_layer.weight', tensor([[-0.1365, -0.5364, -0.3836],
        [ 0.5722, -0.5604, -0.2501],
        [ 0.4147,  0.4546, -0.0468],
        [-0.0063, -0.1399,  0.4483],
        [ 0.2955,  0.5401, -0.3780],
        [ 0.5136, -0.3631, -0.3699],
        [-0.2814,  0.0172,  0.3477],
        [ 0.4181, -0.1767,  0.4972],
        [-0.1799,  0.0568, -0.3362],
        [ 0.2840,  0.5489,  0.6405]])), ('in_layer.bias', tensor([-0.4536, -0.0729,  0.3815, -0.3912, -0.1116,  0.1785, -0.4195, -0.4432,
         0.5161, -0.3506])), ('hd_layer.weight', tensor([[ 4.8242e-02,  1.7043e-01,  2.8597e-01,  5.2063e-02, -7.9224e-02,
         -1.3583e-01,  3.8102e-03,  2.3857e-01, -1.4904e-01, -1.7222e-01],
        [-2.7001e-01, -2.8398e-01, -2.0918e-01, -2.4336e-01, -1.8262e-01,
          3.1319e-01, -1.0663e-01,  9.3339e-02,  2.4816e-01, -1.8231e-01],
        [-2.0293e-01,  2.1843e-01,  1.2676e-01, -1.4377e-01,  1.0281e-01,
         -5.3184e-01, -2.9102e-01,  3.2729e-01, -8.6343e-02,  3.3761e-01],


In [230]:
# 모델 인스턴스에 저장
model2=IrisRegModel()   # 처음부터 다시 돌려야해!
model2.load_state_dict(wbTS)
## 기존에 하던곳 부터 다시 시작

<All keys matched successfully>